In [4]:
import numpy as np
import pandas as pd
import re
#import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#import plotly.tools as tls

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import ClusterCentroids
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
import matplotlib as plt
from random import randint
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [58]:
train= pd.read_excel('/home/ryan/MachineHack/Food_QUalityA_ParticipantsData/Data_Train.xlsx')
test= pd.read_excel('/home/ryan/MachineHack/Food_QUalityA_ParticipantsData/Data_Test.xlsx')

In [59]:
cols_to_drop = [
    'Date',
    'ID'
    ]

In [60]:
zf = train.drop(cols_to_drop, axis=1)

In [61]:
bf = test.drop(cols_to_drop, axis=1)

In [62]:
bf.head(3)

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc
0,34218,2354,2185,GROCERY STORE,17352,id-11235901,id_1890134,81864.0,CANVASS,19.0,Low,locid1145
1,24597,13183,12516,SCHOOL,15536,id-11235901,id_1890134,81871.0,CANVASS,32.0,High,locid16450
2,1779,26434,25220,RESTAURANT,8849,id-11235901,id_1890134,81902.0,CANVASS,NaN,High,locid12916


In [63]:
zf.head(3)

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
0,4744,8123,7715,RESTAURANT,15522,id-11235901,id_1890134,81876.0,CANVASS,33.0,High,locid16406,4
1,2973,12268,11664,GROCERY STORE,3057,id-11235901,id_1890134,81862.0,COMPLAINT,33.0,High,locid878,4
2,18223,1112,969,RESTAURANT,14988,id-11235901,id_1890134,81883.0,CANVASS,NaN,High,locid3368,6


In [64]:
total = zf.isnull().sum().sort_values(ascending=False)
percent_1 = zf.isnull().sum()/zf.isnull().count()*100
percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
missing_data

,Total,%
City,0,0.0
FacilityID,0,0.0
FacilityName,0,0.0
Geo_Loc,0,0.0
Inspection_Results,0,0.0
LicenseNo,0,0.0
LocationID,35,0.0
Reason,0,0.0
RiskLevel,0,0.0
SectionViolations,39068,26.5


In [65]:
total = bf.isnull().sum().sort_values(ascending=False)
percent_1 = bf.isnull().sum()/bf.isnull().count()*100
percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
missing_data

,Total,%
City,0,0.0
FacilityID,0,0.0
FacilityName,0,0.0
Geo_Loc,0,0.0
LicenseNo,0,0.0
LocationID,15,0.0
Reason,0,0.0
RiskLevel,0,0.0
SectionViolations,12976,26.4
State,0,0.0


In [66]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='most_frequent')
dftrain = imp.fit_transform(zf)


In [67]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='most_frequent')
dftest = imp.fit_transform(bf)


In [71]:
dataframetrain = pd.DataFrame.from_records(dftrain)
dataframetest = pd.DataFrame.from_records(dftest)

In [72]:
dataframetrain.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,4744,8123,7715,RESTAURANT,15522,id-11235901,id_1890134,81876.0,CANVASS,33.0,High,locid16406,4
1,2973,12268,11664,GROCERY STORE,3057,id-11235901,id_1890134,81862.0,COMPLAINT,33.0,High,locid878,4
2,18223,1112,969,RESTAURANT,14988,id-11235901,id_1890134,81883.0,CANVASS,32.0,High,locid3368,6


In [73]:

dataframetest.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11
0,34218,2354,2185,GROCERY STORE,17352,id-11235901,id_1890134,81864.0,CANVASS,19.0,Low,locid1145
1,24597,13183,12516,SCHOOL,15536,id-11235901,id_1890134,81871.0,CANVASS,32.0,High,locid16450
2,1779,26434,25220,RESTAURANT,8849,id-11235901,id_1890134,81902.0,CANVASS,32.0,High,locid12916


In [74]:
dataframetrain.rename(columns = { 0:'LicenseNo',1:'FacilityID',2:'FacilityName', 3:'Type',4:'Street',5:'City',6:'State',7:'LocationID',8:'Reason',9:'SectionViolations',10:'RiskLevel',11:'Geo_Loc',12:'Inspection_Results'},inplace=True)
dataframetest.rename(columns = { 0:'LicenseNo',1:'FacilityID',2:'FacilityName', 3:'Type',4:'Street',5:'City',6:'State',7:'LocationID',8:'Reason',9:'SectionViolations',10:'RiskLevel',11:'Geo_Loc'},inplace=True)


In [75]:
dataframetest.head(3)

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc
0,34218,2354,2185,GROCERY STORE,17352,id-11235901,id_1890134,81864.0,CANVASS,19.0,Low,locid1145
1,24597,13183,12516,SCHOOL,15536,id-11235901,id_1890134,81871.0,CANVASS,32.0,High,locid16450
2,1779,26434,25220,RESTAURANT,8849,id-11235901,id_1890134,81902.0,CANVASS,32.0,High,locid12916


In [84]:
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
dataframetrain['Type']= label_encoder.fit_transform(dataframetrain['Type']) 
dataframetrain['Geo_Loc'] = label_encoder.fit_transform(dataframetrain['Geo_Loc'])
dataframetrain['FacilityID'] = label_encoder.fit_transform(dataframetrain['FacilityID'])
dataframetrain['FacilityName'] = label_encoder.fit_transform(dataframetrain['FacilityName'])
dataframetrain['LocationID'] = label_encoder.fit_transform(dataframetrain['LocationID'])
dataframetrain['SectionViolations'] = label_encoder.fit_transform(dataframetrain['SectionViolations'])
dataframetrain['LicenseNo'] = label_encoder.fit_transform(dataframetrain['LicenseNo'])



In [85]:
  
# Encode labels in column 'species'. 
dataframetest['Type']= label_encoder.fit_transform(dataframetest['Type']) 
dataframetest['Geo_Loc'] = label_encoder.fit_transform(dataframetest['Geo_Loc'])
dataframetest['FacilityID'] = label_encoder.fit_transform(dataframetest['FacilityID'])
dataframetest['FacilityName'] = label_encoder.fit_transform(dataframetest['FacilityName'])
dataframetest['LocationID'] = label_encoder.fit_transform(dataframetest['LocationID'])
dataframetest['SectionViolations'] = label_encoder.fit_transform(dataframetest['SectionViolations'])
dataframetest['LicenseNo'] = label_encoder.fit_transform(dataframetest['LicenseNo'])



In [86]:
data =dataframetrain.drop(['Inspection_Results'] , axis =1)

In [87]:
dataframetrain.head(3)
print(data.shape)
print(dataframetrain['Inspection_Results'].shape)

(147443, 12)
(147443,)


In [88]:
smote_nc = SMOTENC(categorical_features=[3,5,6,8,9,10,], random_state=0) 

X_resampled, y_resampled = smote_nc.fit_resample(data,dataframetrain['Inspection_Results'])
print(sorted(Counter(y_resampled).items()))

[(0, 79453), (1, 79453), (2, 79453), (3, 79453), (4, 79453), (5, 79453), (6, 79453)]


In [89]:
# ndataframe

In [91]:
 ndataframe = pd.concat([X_resampled,y_resampled] , axis =1)

In [92]:
ndataframe.head(3)

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc,Inspection_Results
0,4203,7549,7157,326,15522,id-11235901,id_1890134,69,CANVASS,32,High,6889,4
1,2635,11435,10865,182,3057,id-11235901,id_1890134,55,COMPLAINT,32,High,14998,4
2,16649,1023,879,326,14988,id-11235901,id_1890134,75,CANVASS,31,High,9140,6


In [96]:
dataframetest.head(3)

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc
0,20557,1463,1356,127,17352,id-11235901,id_1890134,41,CANVASS,18,Low,1341
1,15355,8620,8222,256,15536,id-11235901,id_1890134,48,CANVASS,31,High,5812
2,1006,17554,16786,234,8849,id-11235901,id_1890134,76,CANVASS,31,High,2658


In [97]:
ndataframe = pd.get_dummies(ndataframe, prefix="RiskLevel",columns=['RiskLevel'])
ndataframe = pd.get_dummies(ndataframe, prefix="City",columns=['City'])
ndataframe = pd.get_dummies(ndataframe, prefix="State",columns=['State'])
ndataframe = pd.get_dummies(ndataframe, prefix="Reason",columns=['Reason'])

In [98]:
dataframetest = pd.get_dummies(dataframetest, prefix="RiskLevel",columns=['RiskLevel'])
dataframetest = pd.get_dummies(dataframetest, prefix="City",columns=['City'])
dataframetest = pd.get_dummies(dataframetest, prefix="State",columns=['State'])
dataframetest = pd.get_dummies(dataframetest, prefix="Reason",columns=['Reason'])

In [105]:
ndataframe.columns
#dataframetest.columns

Index(['LicenseNo', 'FacilityID', 'FacilityName', 'Type', 'Street',
       'LocationID', 'SectionViolations', 'Geo_Loc', 'Inspection_Results',
       'RiskLevel_High', 'RiskLevel_Low', 'RiskLevel_Medium',
       'RiskLevel_Uncertain', 'City_id-11235901', 'City_id-11275913',
       'State_id_1890134', 'State_id_1890135', 'Reason_CANVASS',
       'Reason_CANVASS RE-INSPECTION', 'Reason_COMPLAINT',
       'Reason_COMPLAINT RE-INSPECTION', 'Reason_COMPLAINT-FIRE',
       'Reason_CONSULTATION', 'Reason_LICENSE', 'Reason_LICENSE RE-INSPECTION',
       'Reason_LICENSE-TASK FORCE', 'Reason_OUT OF BUSINESS',
       'Reason_RECENT INSPECTION', 'Reason_SHORT FORM COMPLAINT',
       'Reason_SHORT FORM FIRE-COMPLAINT', 'Reason_SUSPECTED FOOD POISONING',
       'Reason_SUSPECTED FOOD POISONING RE-INSPECTION', 'Reason_TAG REMOVAL',
       'Reason_TASK FORCE LIQUOR 1475'],
      dtype='object')

In [106]:
Y = ndataframe['Inspection_Results']
X = ndataframe.drop('Inspection_Results', axis =1)


In [ ]:
#####################checking for data leakage and applying some maths ###################

In [109]:
train =X.copy()
test = dataframetest.copy()

In [110]:
train.describe()

,LicenseNo,FacilityID,FacilityName,Type,Street,LocationID,SectionViolations,Geo_Loc,RiskLevel_High,RiskLevel_Low,...,Reason_LICENSE RE-INSPECTION,Reason_LICENSE-TASK FORCE,Reason_OUT OF BUSINESS,Reason_RECENT INSPECTION,Reason_SHORT FORM COMPLAINT,Reason_SHORT FORM FIRE-COMPLAINT,Reason_SUSPECTED FOOD POISONING,Reason_SUSPECTED FOOD POISONING RE-INSPECTION,Reason_TAG REMOVAL,Reason_TASK FORCE LIQUOR 1475
count,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,...,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000,556171.000000
mean,17000.988302,12748.852376,12248.705760,302.672743,8762.641817,65.189967,25.375210,8419.720426,0.711181,0.112327,...,0.015610,0.001334,0.000606,0.000451,0.012534,0.000138,0.001266,0.000264,0.001063,0.000466
std,10167.136245,7219.245617,6816.667604,61.249299,5159.693825,15.077612,10.690548,4693.787220,0.453214,0.315769,...,0.123962,0.036501,0.024608,0.021239,0.111251,0.011766,0.035556,0.016255,0.032581,0.021575
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8288.000000,6486.000000,6381.000000,326.000000,4264.000000,52.000000,20.000000,4320.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,17003.000000,12605.000000,12414.000000,326.000000,8540.000000,63.000000,31.000000,8392.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,26027.000000,19121.000000,18047.000000,326.000000,13347.000000,77.000000,31.000000,12821.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,34383.000000,25534.000000,24347.000000,408.000000,17891.000000,103.000000,60.000000,16315.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [111]:
test.describe()

,LicenseNo,FacilityID,FacilityName,Type,Street,LocationID,SectionViolations,Geo_Loc,RiskLevel_High,RiskLevel_Low,...,Reason_LICENSE RE-INSPECTION,Reason_LICENSE-TASK FORCE,Reason_OUT OF BUSINESS,Reason_RECENT INSPECTION,Reason_SHORT FORM COMPLAINT,Reason_SHORT FORM FIRE-COMPLAINT,Reason_SUSPECTED FOOD POISONING,Reason_SUSPECTED FOOD POISONING RE-INSPECTION,Reason_TAG REMOVAL,Reason_TASK FORCE LIQUOR 1475
count,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,...,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000,49148.000000
mean,10746.064275,9193.152295,8792.277204,206.815069,8654.251343,50.094592,25.192175,7036.248433,0.718035,0.087043,...,0.045617,0.003276,0.001729,0.001811,0.033979,0.000732,0.004517,0.000895,0.002991,0.001323
std,6686.695302,5181.857377,4950.033059,57.615008,5272.693487,16.562966,11.354419,3955.365854,0.449961,0.281901,...,0.208656,0.057142,0.041551,0.042516,0.181177,0.027055,0.067057,0.029908,0.054608,0.036343
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4940.750000,4775.000000,4544.000000,234.000000,4027.000000,36.000000,17.000000,3620.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10302.000000,9294.000000,8969.500000,234.000000,8505.000000,47.000000,31.000000,7055.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,16329.000000,13708.000000,13074.000000,234.000000,13329.250000,64.000000,31.000000,10648.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,22524.000000,18103.000000,17325.000000,300.000000,17890.000000,85.000000,60.000000,13644.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [112]:
clf1=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.5,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.6, verbosity=1)


In [117]:
clf2 = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [118]:
clf3 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=11, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [119]:
clf4 = BaggingClassifier(base_estimator=clf3, n_estimators=100, max_samples=1.0, max_features=1.0, 
                         bootstrap=True,
                         bootstrap_features=False, oob_score=True, warm_start=False,
                         n_jobs=-1, random_state=None, verbose=0)
  

In [120]:
sclf = StackingClassifier(classifiers=[clf2, clf3,clf4],meta_classifier=clf1)

In [121]:
clf4.fit(X,Y)
f4 = clf4.predict(test)

In [122]:
sclf.fit(X,Y)
sf = sclf.predict(test)

In [123]:
dataclf4 = pd.DataFrame(f4.tolist(),columns=['Ins'])
datasclf = pd.DataFrame(sf.tolist(),columns=['Ins'])

In [129]:
dataclf4

,Ins
0,1
1,6
2,6
3,6
4,4
...,...
49143,6
49144,4
49145,4
49146,5


In [125]:
datasclf.head(3)

,Ins
0,1
1,5
2,6


In [130]:
g4 = pd.get_dummies(dataclf4, prefix="",columns=['Ins'])
sg = pd.get_dummies(datasclf, prefix="",columns=['Ins'])

In [132]:
sg

,_0,_1,_2,_3,_4,_5,_6
0,0,1,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...
49143,0,0,0,0,0,0,1
49144,0,0,0,0,1,0,0
49145,0,0,0,0,1,0,0
49146,0,0,0,0,0,1,0


In [133]:
g4.to_excel(excel_writer = "/home/ryan/MachineHack/Food_QUalityA_ParticipantsData/submission_g4NEW.xlsx")

In [134]:
sg.to_excel(excel_writer = "/home/ryan/MachineHack/Food_QUalityA_ParticipantsData/submission_sgNEW.xlsx")